In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

In [6]:
# load a file that uses , as delimiter
def load_file(path, names):
    if not path.is_file():
        raise FileNotFoundError(str(path))
    data = pd.read_csv(path, sep=",", names=names, header=None)
    return data

In [7]:
def load_df():
    cols = ["state", "emission"]
    file = Path.cwd() / "data.txt"
    return load_file(file, cols)

In [13]:
data = load_df()
data

,state,emission
0,foggy,no
1,foggy,no
2,foggy,no
3,rainy,yes
4,sunny,no
...,...,...
995,sunny,no
996,foggy,no
997,rainy,yes
998,foggy,yes
